In [13]:
# Add execute permissions to binaries
!chmod +x dssp
!chmod +x DAlphaBall.gcc

# Verify permissions
!ls -l dssp
!ls -l DAlphaBall.gcc

-rwxr-xr-x@ 1 satishgaurav  staff  877904 May  4 21:24 dssp
-rwxr-xr-x@ 1 satishgaurav  staff  345824 May  4 21:24 DAlphaBall.gcc


In [14]:
from IPython.display import display, Markdown

In [16]:
####################################
################ BioPython functions
####################################
### Import dependencies
import os
import math
import numpy as np
from collections import defaultdict
from scipy.spatial import cKDTree
from Bio import BiopythonWarning
from Bio.PDB import PDBParser, DSSP, Selection, Polypeptide, PDBIO, Select, Chain, Superimposer
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB.Selection import unfold_entities
from Bio.PDB.Polypeptide import is_aa

# analyze sequence composition of design
def validate_design_sequence(sequence, num_clashes, advanced_settings):
    note_array = []

    # Check if protein contains clashes after relaxation
    if num_clashes > 0:
        note_array.append('Relaxed structure contains clashes.')

    # Check if the sequence contains disallowed amino acids
    if advanced_settings["omit_AAs"]:
        restricted_AAs = advanced_settings["omit_AAs"].split(',')
        for restricted_AA in restricted_AAs:
            if restricted_AA in sequence:
                note_array.append('Contains: '+restricted_AA+'!')

    # Analyze the protein
    analysis = ProteinAnalysis(sequence)

    # Calculate the reduced extinction coefficient per 1% solution
    extinction_coefficient_reduced = analysis.molar_extinction_coefficient()[0]
    molecular_weight = round(analysis.molecular_weight() / 1000, 2)
    extinction_coefficient_reduced_1 = round(extinction_coefficient_reduced / molecular_weight * 0.01, 2)

    # Check if the absorption is high enough
    if extinction_coefficient_reduced_1 <= 2:
        note_array.append(f'Absorption value is {extinction_coefficient_reduced_1}, consider adding tryptophane to design.')

    # Join the notes into a single string
    notes = ' '.join(note_array)

    return notes

# temporary function, calculate RMSD of input PDB and trajectory target
def target_pdb_rmsd(trajectory_pdb, starting_pdb, chain_ids_string):
    # Parse the PDB files
    parser = PDBParser(QUIET=True)
    structure_trajectory = parser.get_structure('trajectory', trajectory_pdb)
    structure_starting = parser.get_structure('starting', starting_pdb)
    
    # Extract chain A from trajectory_pdb
    chain_trajectory = structure_trajectory[0]['A']
    
    # Extract the specified chains from starting_pdb
    chain_ids = chain_ids_string.split(',')
    residues_starting = []
    for chain_id in chain_ids:
        chain_id = chain_id.strip()
        chain = structure_starting[0][chain_id]
        for residue in chain:
            if is_aa(residue, standard=True):
                residues_starting.append(residue)
    
    # Extract residues from chain A in trajectory_pdb
    residues_trajectory = [residue for residue in chain_trajectory if is_aa(residue, standard=True)]
    
    # Ensure that both structures have the same number of residues
    min_length = min(len(residues_starting), len(residues_trajectory))
    residues_starting = residues_starting[:min_length]
    residues_trajectory = residues_trajectory[:min_length]
    
    # Collect CA atoms from the two sets of residues
    atoms_starting = [residue['CA'] for residue in residues_starting if 'CA' in residue]
    atoms_trajectory = [residue['CA'] for residue in residues_trajectory if 'CA' in residue]
    
    # Calculate RMSD using structural alignment
    sup = Superimposer()
    sup.set_atoms(atoms_starting, atoms_trajectory)
    rmsd = sup.rms
    
    return round(rmsd, 2)

# detect C alpha clashes for deformed trajectories
def calculate_clash_score(pdb_file, threshold=2.4, only_ca=False):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    atoms = []
    atom_info = []  # Detailed atom info for debugging and processing

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element == 'H':  # Skip hydrogen atoms
                        continue
                    if only_ca and atom.get_name() != 'CA':
                        continue
                    atoms.append(atom.coord)
                    atom_info.append((chain.id, residue.id[1], atom.get_name(), atom.coord))

    tree = cKDTree(atoms)
    pairs = tree.query_pairs(threshold)

    valid_pairs = set()
    for (i, j) in pairs:
        chain_i, res_i, name_i, coord_i = atom_info[i]
        chain_j, res_j, name_j, coord_j = atom_info[j]

        # Exclude clashes within the same residue
        if chain_i == chain_j and res_i == res_j:
            continue

        # Exclude directly sequential residues in the same chain for all atoms
        if chain_i == chain_j and abs(res_i - res_j) == 1:
            continue

        # If calculating sidechain clashes, only consider clashes between different chains
        if not only_ca and chain_i == chain_j:
            continue

        valid_pairs.add((i, j))

    return len(valid_pairs)

three_to_one_map = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

# identify interacting residues at the binder interface
def hotspot_residues(trajectory_pdb, binder_chain="B", atom_distance_cutoff=4.0):
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", trajectory_pdb)

    # Get the specified chain
    binder_atoms = Selection.unfold_entities(structure[0][binder_chain], 'A')
    binder_coords = np.array([atom.coord for atom in binder_atoms])

    # Get atoms and coords for the target chain
    target_atoms = Selection.unfold_entities(structure[0]['A'], 'A')
    target_coords = np.array([atom.coord for atom in target_atoms])

    # Build KD trees for both chains
    binder_tree = cKDTree(binder_coords)
    target_tree = cKDTree(target_coords)

    # Prepare to collect interacting residues
    interacting_residues = {}

    # Query the tree for pairs of atoms within the distance cutoff
    pairs = binder_tree.query_ball_tree(target_tree, atom_distance_cutoff)

    # Process each binder atom's interactions
    for binder_idx, close_indices in enumerate(pairs):
        binder_residue = binder_atoms[binder_idx].get_parent()
        binder_resname = binder_residue.get_resname()

        # Convert three-letter code to single-letter code using the manual dictionary
        if binder_resname in three_to_one_map:
            aa_single_letter = three_to_one_map[binder_resname]
            for close_idx in close_indices:
                target_residue = target_atoms[close_idx].get_parent()
                interacting_residues[binder_residue.id[1]] = aa_single_letter

    return interacting_residues

# calculate secondary structure percentage of design
def calc_ss_percentage(pdb_file, advanced_settings, chain_id="B", atom_distance_cutoff=4.0):
    # Parse the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]  # Consider only the first model in the structure

    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')

    # Prepare to count residues
    ss_counts = defaultdict(int)
    ss_interface_counts = defaultdict(int)
    plddts_interface = []
    plddts_ss = []

    # Get chain and interacting residues once
    chain = model[chain_id]
    interacting_residues = set(hotspot_residues(pdb_file, chain_id, atom_distance_cutoff).keys())

    for residue in chain:
        residue_id = residue.id[1]
        if (chain_id, residue_id) in dssp:
            ss = dssp[(chain_id, residue_id)][2]  # Get the secondary structure
            ss_type = 'loop'
            if ss in ['H', 'G', 'I']:
                ss_type = 'helix'
            elif ss == 'E':
                ss_type = 'sheet'

            ss_counts[ss_type] += 1

            if ss_type != 'loop':
                # calculate secondary structure normalised pLDDT
                avg_plddt_ss = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_ss.append(avg_plddt_ss)

            if residue_id in interacting_residues:
                ss_interface_counts[ss_type] += 1

                # calculate interface pLDDT
                avg_plddt_residue = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_interface.append(avg_plddt_residue)

    # Calculate percentages
    total_residues = sum(ss_counts.values())
    total_interface_residues = sum(ss_interface_counts.values())

    percentages = calculate_percentages(total_residues, ss_counts['helix'], ss_counts['sheet'])
    interface_percentages = calculate_percentages(total_interface_residues, ss_interface_counts['helix'], ss_interface_counts['sheet'])

    i_plddt = round(sum(plddts_interface) / len(plddts_interface) / 100, 2) if plddts_interface else 0
    ss_plddt = round(sum(plddts_ss) / len(plddts_ss) / 100, 2) if plddts_ss else 0

    return (*percentages, *interface_percentages, i_plddt, ss_plddt)

def calculate_percentages(total, helix, sheet):
    helix_percentage = round((helix / total) * 100,2) if total > 0 else 0
    sheet_percentage = round((sheet / total) * 100,2) if total > 0 else 0
    loop_percentage = round(((total - helix - sheet) / total) * 100,2) if total > 0 else 0


    return helix_percentage, sheet_percentage, loop_percentage

In [17]:
advanced_settings = {
    "dssp_path": "/usr/local/bin/mkdssp",
    "omit_AAs": "C",
}
pdb_file = "./../target/9bk6.pdb"
calc_ss_percentage(pdb_file=pdb_file, advanced_settings=advanced_settings)

(0.0, 50.0, 50.0, 0.0, 55.56, 44.44, 0.43, 0.43)

In [19]:
import pandas as pd

AA_PROPERTIES = {
        "ALA": "nonpolar",
        "ARG": "positive",
        "ASN": "polar",
        "ASP": "negative",
        "CYS": "polar",
        "GLN": "polar",
        "GLU": "negative",
        "GLY": "nonpolar",
        "HIS": "positive",
        "ILE": "nonpolar",
        "LEU": "nonpolar",
        "LYS": "positive",
        "MET": "nonpolar",
        "PHE": "nonpolar",
        "PRO": "nonpolar",
        "SER": "polar",
        "THR": "polar",
        "TRP": "nonpolar",
        "TYR": "polar",
        "VAL": "nonpolar",
        "SEC": "polar"
    }

def create_structure_df(pdb_file, advanced_settings, chain_id=None, atom_distance_cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]
    
    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')
    
    # Prepare data for DataFrame
    data = []
    
    # Process either specific chain or all chains
    chains = [model[chain_id]] if chain_id else model.get_chains()
    
    for chain in chains:
        current_chain_id = chain.id
        # Get interacting residues if chain_id is specified
        interacting_residues = set()
        if current_chain_id:
            interacting_residues = set(hotspot_residues(pdb_file, current_chain_id, atom_distance_cutoff).keys())
        
        for residue in chain:
            residue_id = residue.id[1]
            res_name = residue.get_resname()
            
            if (current_chain_id, residue_id) in dssp:
                dssp_data = dssp[(current_chain_id, residue_id)]
                ss = dssp_data[2]
                acc = dssp_data[3]
                phi = dssp_data[4]
                psi = dssp_data[5]
                
                ss_type = 'loop'
                if ss in ['H', 'G', 'I']:
                    ss_type = 'helix'
                elif ss == 'E':
                    ss_type = 'sheet'
                    
                avg_plddt = sum(atom.bfactor for atom in residue) / len(residue) / 100
                
                data.append({
                    'chain_id': current_chain_id,
                    'residue_id': residue_id,
                    'residue_name': res_name,
                    'property': AA_PROPERTIES.get(res_name, 'unknown'),
                    'ss_type': ss_type,
                    'ss_code': ss,
                    'accessibility': acc,
                    'phi': phi,
                    'psi': psi,
                    'plddt': round(avg_plddt, 2),
                    'is_interface': residue_id in interacting_residues if chain_id else None
                })
    
    return pd.DataFrame(data)


# Example usage:
pdb_file = "./../target/9bk6.pdb"
df = create_structure_df(pdb_file, advanced_settings)
df.head()

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,3,GLY,nonpolar,loop,-,1.000000,360.0,178.5,0.64,None
1,A,4,GLY,nonpolar,loop,-,0.464286,-107.8,-117.5,0.55,None
2,A,5,THR,polar,loop,-,0.366197,-85.9,163.0,0.42,None
3,A,6,PRO,nonpolar,helix,H,0.250000,-55.6,-35.0,0.44,None
4,A,7,GLU,negative,helix,H,0.515464,-71.4,-33.1,0.50,None


In [20]:
df.tail(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
151,B,151,TYR,polar,sheet,E,0.094595,-113.1,132.1,0.40,None
152,B,152,MET,nonpolar,sheet,E,0.430851,-119.6,125.4,0.44,None
153,B,153,CYS,polar,sheet,E,0.274074,-123.2,151.6,0.41,None
154,B,154,CYS,polar,sheet,E,0.162963,-161.3,163.1,0.36,None
155,B,155,ASN,polar,loop,-,0.732484,-129.7,30.0,0.42,None
156,B,156,THR,polar,loop,S,0.577465,-120.9,167.6,0.46,None
157,B,157,ASP,negative,loop,T,0.564417,-66.2,128.8,0.47,None
158,B,158,LYS,positive,loop,T,0.468293,57.4,31.4,0.46,None
159,B,159,CYS,polar,loop,-,0.266667,-80.6,-26.6,0.36,None
160,B,160,ASN,polar,loop,-,0.038217,-87.8,360.0,0.38,None


In [21]:
df.head(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,3,GLY,nonpolar,loop,-,1.000000,360.0,178.5,0.64,None
1,A,4,GLY,nonpolar,loop,-,0.464286,-107.8,-117.5,0.55,None
2,A,5,THR,polar,loop,-,0.366197,-85.9,163.0,0.42,None
3,A,6,PRO,nonpolar,helix,H,0.250000,-55.6,-35.0,0.44,None
4,A,7,GLU,negative,helix,H,0.515464,-71.4,-33.1,0.50,None
5,A,8,GLU,negative,helix,H,0.608247,-75.0,-37.5,0.44,None
6,A,9,ARG,positive,helix,H,0.366935,-66.0,-33.0,0.43,None
7,A,10,LEU,nonpolar,helix,H,0.189024,-70.2,-38.7,0.39,None
8,A,11,ALA,nonpolar,helix,H,0.556604,-66.8,-32.2,0.43,None
9,A,12,GLN,polar,helix,H,0.424242,-68.1,-41.1,0.46,None


In [22]:
df_target = df[df['chain_id'] == 'B']
df_target = df_target.reset_index(drop=True)
df_target.index = range(1, len(df_target) + 1)

In [23]:
df_target.to_csv('9bk6_chain_B_secondary_structure_information.csv', index=False)

In [24]:
df_target[df_target['property'] == 'nonpolar']

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
1,B,101,LEU,nonpolar,loop,-,0.371951,360.0,128.4,0.41,None
6,B,106,LEU,nonpolar,loop,S,0.451220,-62.8,-30.9,0.37,None
7,B,107,ILE,nonpolar,loop,S,0.698225,-97.5,107.8,0.43,None
8,B,108,PRO,nonpolar,loop,T,0.676471,-70.4,155.3,0.51,None
9,B,109,PRO,nonpolar,loop,T,0.772059,-95.9,29.2,0.50,None
10,B,110,PHE,nonpolar,loop,-,0.664975,-84.7,150.1,0.44,None
11,B,111,TRP,nonpolar,sheet,E,0.555066,-126.2,173.9,0.53,None
15,B,115,PRO,nonpolar,loop,-,0.375000,-76.9,176.1,0.52,None
17,B,117,GLY,nonpolar,loop,T,0.619048,106.9,-21.3,0.60,None
20,B,120,LEU,nonpolar,sheet,E,0.262195,-145.5,145.8,0.41,None


In [25]:
display(Markdown(df_target[df_target['property'] == 'nonpolar'].sort_values('accessibility', ascending=False).to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 31 | B          |          131 | MET            | nonpolar   | sheet     | E         |      0.87234    | -102.7 |   11.5 |    0.65 |                |
|  9 | B          |          109 | PRO            | nonpolar   | loop      | T         |      0.772059   |  -95.9 |   29.2 |    0.5  |                |
|  7 | B          |          107 | ILE            | nonpolar   | loop      | S         |      0.698225   |  -97.5 |  107.8 |    0.43 |                |
| 41 | B          |          141 | VAL            | nonpolar   | loop      | S         |      0.683099   | -121.8 |  140.2 |    0.49 |                |
|  8 | B          |          108 | PRO            | nonpolar   | loop      | T         |      0.676471   |  -70.4 |  155.3 |    0.51 |                |
| 10 | B          |          110 | PHE            | nonpolar   | loop      | -         |      0.664975   |  -84.7 |  150.1 |    0.44 |                |
| 28 | B          |          128 | ALA            | nonpolar   | loop      | T         |      0.650943   |  -97   |  -11.3 |    0.59 |                |
| 32 | B          |          132 | VAL            | nonpolar   | sheet     | E         |      0.647887   | -123   |   90.9 |    0.53 |                |
| 17 | B          |          117 | GLY            | nonpolar   | loop      | T         |      0.619048   |  106.9 |  -21.3 |    0.6  |                |
| 30 | B          |          130 | PRO            | nonpolar   | sheet     | E         |      0.558824   |  -72.4 |  -11.8 |    0.55 |                |
| 11 | B          |          111 | TRP            | nonpolar   | sheet     | E         |      0.555066   | -126.2 |  173.9 |    0.53 |                |
|  6 | B          |          106 | LEU            | nonpolar   | loop      | S         |      0.45122    |  -62.8 |  -30.9 |    0.37 |                |
| 52 | B          |          152 | MET            | nonpolar   | sheet     | E         |      0.430851   | -119.6 |  125.4 |    0.44 |                |
| 15 | B          |          115 | PRO            | nonpolar   | loop      | -         |      0.375      |  -76.9 |  176.1 |    0.52 |                |
|  1 | B          |          101 | LEU            | nonpolar   | loop      | -         |      0.371951   |  360   |  128.4 |    0.41 |                |
| 33 | B          |          133 | PRO            | nonpolar   | sheet     | E         |      0.316176   |  -67   |  135.5 |    0.45 |                |
| 29 | B          |          129 | ALA            | nonpolar   | sheet     | E         |      0.292453   | -129.2 |   85.3 |    0.58 |                |
| 20 | B          |          120 | LEU            | nonpolar   | sheet     | E         |      0.262195   | -145.5 |  145.8 |    0.41 |                |
| 34 | B          |          134 | VAL            | nonpolar   | sheet     | E         |      0.204225   |  -97.4 |  -42.9 |    0.39 |                |
| 43 | B          |          143 | PRO            | nonpolar   | loop      | P         |      0.191176   |  -68.8 |  143.1 |    0.42 |                |
| 26 | B          |          126 | MET            | nonpolar   | sheet     | E         |      0.0797872  |  -68.4 |  151   |    0.45 |                |
| 37 | B          |          137 | GLY            | nonpolar   | sheet     | E         |      0.0595238  |  167.3 | -156.3 |    0.36 |                |
| 39 | B          |          139 | ILE            | nonpolar   | sheet     | E         |      0.0295858  | -162.2 |  157.9 |    0.42 |                |
| 24 | B          |          124 | MET            | nonpolar   | sheet     | E         |      0.0106383  | -115.3 |  125   |    0.37 |                |
| 47 | B          |          147 | LEU            | nonpolar   | loop      | S         |      0.00609756 |  -66.9 |  -25.5 |    0.39 |                |
| 49 | B          |          149 | ILE            | nonpolar   | sheet     | E         |      0.00591716 | -125.3 |  135.9 |    0.35 |                |
| 48 | B          |          148 | LEU            | nonpolar   | loop      | S         |      0          | -113   |  -18.6 |    0.36 |                |

In [26]:
display(Markdown(df_target.sort_values('accessibility', ascending=False).to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 16 | B          |          116 | LYS            | positive   | loop      | T         |      0.95122    |  -70.1 |  132.1 |    0.6  |                |
| 31 | B          |          131 | MET            | nonpolar   | sheet     | E         |      0.87234    | -102.7 |   11.5 |    0.65 |                |
|  9 | B          |          109 | PRO            | nonpolar   | loop      | T         |      0.772059   |  -95.9 |   29.2 |    0.5  |                |
| 19 | B          |          119 | ASN            | polar      | loop      | -         |      0.738854   | -107.5 |   30   |    0.56 |                |
| 55 | B          |          155 | ASN            | polar      | loop      | -         |      0.732484   | -129.7 |   30   |    0.42 |                |
|  7 | B          |          107 | ILE            | nonpolar   | loop      | S         |      0.698225   |  -97.5 |  107.8 |    0.43 |                |
| 41 | B          |          141 | VAL            | nonpolar   | loop      | S         |      0.683099   | -121.8 |  140.2 |    0.49 |                |
|  8 | B          |          108 | PRO            | nonpolar   | loop      | T         |      0.676471   |  -70.4 |  155.3 |    0.51 |                |
| 10 | B          |          110 | PHE            | nonpolar   | loop      | -         |      0.664975   |  -84.7 |  150.1 |    0.44 |                |
| 28 | B          |          128 | ALA            | nonpolar   | loop      | T         |      0.650943   |  -97   |  -11.3 |    0.59 |                |
| 32 | B          |          132 | VAL            | nonpolar   | sheet     | E         |      0.647887   | -123   |   90.9 |    0.53 |                |
| 17 | B          |          117 | GLY            | nonpolar   | loop      | T         |      0.619048   |  106.9 |  -21.3 |    0.6  |                |
| 40 | B          |          140 | ASP            | negative   | loop      | S         |      0.595092   |  -88.3 |  -51.2 |    0.55 |                |
| 13 | B          |          113 | THR            | polar      | sheet     | E         |      0.577465   |  -69.8 |  112.2 |    0.4  |                |
| 56 | B          |          156 | THR            | polar      | loop      | S         |      0.577465   | -120.9 |  167.6 |    0.46 |                |
| 57 | B          |          157 | ASP            | negative   | loop      | T         |      0.564417   |  -66.2 |  128.8 |    0.47 |                |
| 30 | B          |          130 | PRO            | nonpolar   | sheet     | E         |      0.558824   |  -72.4 |  -11.8 |    0.55 |                |
| 11 | B          |          111 | TRP            | nonpolar   | sheet     | E         |      0.555066   | -126.2 |  173.9 |    0.53 |                |
| 58 | B          |          158 | LYS            | positive   | loop      | T         |      0.468293   |   57.4 |   31.4 |    0.46 |                |
|  6 | B          |          106 | LEU            | nonpolar   | loop      | S         |      0.45122    |  -62.8 |  -30.9 |    0.37 |                |
| 12 | B          |          112 | LYS            | positive   | sheet     | E         |      0.434146   | -139.2 |  122.6 |    0.46 |                |
| 52 | B          |          152 | MET            | nonpolar   | sheet     | E         |      0.430851   | -119.6 |  125.4 |    0.44 |                |
| 50 | B          |          150 | LYS            | positive   | sheet     | E         |      0.385366   | -113.9 |  132.1 |    0.46 |                |
|  2 | B          |          102 | LYS            | positive   | sheet     | E         |      0.37561    | -112.8 |  127.3 |    0.47 |                |
| 15 | B          |          115 | PRO            | nonpolar   | loop      | -         |      0.375      |  -76.9 |  176.1 |    0.52 |                |
|  1 | B          |          101 | LEU            | nonpolar   | loop      | -         |      0.371951   |  360   |  128.4 |    0.41 |                |
| 44 | B          |          144 | LYS            | positive   | loop      | P         |      0.346341   |  -72.8 |  135.3 |    0.48 |                |
| 42 | B          |          142 | CYS            | polar      | loop      | P         |      0.325926   |  -69.6 |  125.3 |    0.46 |                |
|  5 | B          |          105 | GLN            | polar      | loop      | -         |      0.318182   |  -92.2 |  168.9 |    0.39 |                |
| 33 | B          |          133 | PRO            | nonpolar   | sheet     | E         |      0.316176   |  -67   |  135.5 |    0.45 |                |
| 29 | B          |          129 | ALA            | nonpolar   | sheet     | E         |      0.292453   | -129.2 |   85.3 |    0.58 |                |
| 53 | B          |          153 | CYS            | polar      | sheet     | E         |      0.274074   | -123.2 |  151.6 |    0.41 |                |
| 59 | B          |          159 | CYS            | polar      | loop      | -         |      0.266667   |  -80.6 |  -26.6 |    0.36 |                |
| 20 | B          |          120 | LEU            | nonpolar   | sheet     | E         |      0.262195   | -145.5 |  145.8 |    0.41 |                |
| 36 | B          |          136 | ARG            | positive   | sheet     | E         |      0.233871   | -165.1 |  118.4 |    0.4  |                |
| 23 | B          |          123 | LYS            | positive   | sheet     | E         |      0.229268   | -129   |  136.6 |    0.42 |                |
| 34 | B          |          134 | VAL            | nonpolar   | sheet     | E         |      0.204225   |  -97.4 |  -42.9 |    0.39 |                |
| 46 | B          |          146 | SER            | polar      | loop      | -         |      0.2        | -125.9 | -170.1 |    0.36 |                |
| 43 | B          |          143 | PRO            | nonpolar   | loop      | P         |      0.191176   |  -68.8 |  143.1 |    0.42 |                |
| 18 | B          |          118 | LYS            | positive   | loop      | -         |      0.17561    | -109.1 |   94.5 |    0.51 |                |
|  4 | B          |          104 | ASN            | polar      | sheet     | E         |      0.171975   |  -63.4 |  141.3 |    0.36 |                |
| 35 | B          |          135 | LYS            | positive   | sheet     | E         |      0.165854   | -135.1 |  150.8 |    0.41 |                |
| 54 | B          |          154 | CYS            | polar      | sheet     | E         |      0.162963   | -161.3 |  163.1 |    0.36 |                |
| 27 | B          |          127 | ARG            | positive   | loop      | T         |      0.145161   |  -60.7 |  -25.4 |    0.47 |                |
| 38 | B          |          138 | CYS            | polar      | sheet     | E         |      0.118519   |  -77.9 |  155.3 |    0.38 |                |
| 51 | B          |          151 | TYR            | polar      | sheet     | E         |      0.0945946  | -113.1 |  132.1 |    0.4  |                |
| 25 | B          |          125 | THR            | polar      | sheet     | E         |      0.084507   | -138   |  159.7 |    0.39 |                |
| 26 | B          |          126 | MET            | nonpolar   | sheet     | E         |      0.0797872  |  -68.4 |  151   |    0.45 |                |
| 45 | B          |          145 | SER            | polar      | loop      | P         |      0.0692308  |  -65.4 |  156.8 |    0.38 |                |
| 14 | B          |          114 | CYS            | polar      | loop      | -         |      0.0666667  |  -66.1 |  144.5 |    0.42 |                |
| 37 | B          |          137 | GLY            | nonpolar   | sheet     | E         |      0.0595238  |  167.3 | -156.3 |    0.36 |                |
| 60 | B          |          160 | ASN            | polar      | loop      | -         |      0.0382166  |  -87.8 |  360   |    0.38 |                |
| 39 | B          |          139 | ILE            | nonpolar   | sheet     | E         |      0.0295858  | -162.2 |  157.9 |    0.42 |                |
| 22 | B          |          122 | TYR            | polar      | sheet     | E         |      0.0225225  | -145   |  150.2 |    0.38 |                |
| 24 | B          |          124 | MET            | nonpolar   | sheet     | E         |      0.0106383  | -115.3 |  125   |    0.37 |                |
| 47 | B          |          147 | LEU            | nonpolar   | loop      | S         |      0.00609756 |  -66.9 |  -25.5 |    0.39 |                |
| 49 | B          |          149 | ILE            | nonpolar   | sheet     | E         |      0.00591716 | -125.3 |  135.9 |    0.35 |                |
| 48 | B          |          148 | LEU            | nonpolar   | loop      | S         |      0          | -113   |  -18.6 |    0.36 |                |
| 21 | B          |          121 | CYS            | polar      | sheet     | E         |      0          |  -98.4 |  150   |    0.34 |                |
|  3 | B          |          103 | CYS            | polar      | sheet     | E         |      0          | -119.8 |  138.6 |    0.33 |                |

In [27]:
display(Markdown(df_target.to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
|  1 | B          |          101 | LEU            | nonpolar   | loop      | -         |      0.371951   |  360   |  128.4 |    0.41 |                |
|  2 | B          |          102 | LYS            | positive   | sheet     | E         |      0.37561    | -112.8 |  127.3 |    0.47 |                |
|  3 | B          |          103 | CYS            | polar      | sheet     | E         |      0          | -119.8 |  138.6 |    0.33 |                |
|  4 | B          |          104 | ASN            | polar      | sheet     | E         |      0.171975   |  -63.4 |  141.3 |    0.36 |                |
|  5 | B          |          105 | GLN            | polar      | loop      | -         |      0.318182   |  -92.2 |  168.9 |    0.39 |                |
|  6 | B          |          106 | LEU            | nonpolar   | loop      | S         |      0.45122    |  -62.8 |  -30.9 |    0.37 |                |
|  7 | B          |          107 | ILE            | nonpolar   | loop      | S         |      0.698225   |  -97.5 |  107.8 |    0.43 |                |
|  8 | B          |          108 | PRO            | nonpolar   | loop      | T         |      0.676471   |  -70.4 |  155.3 |    0.51 |                |
|  9 | B          |          109 | PRO            | nonpolar   | loop      | T         |      0.772059   |  -95.9 |   29.2 |    0.5  |                |
| 10 | B          |          110 | PHE            | nonpolar   | loop      | -         |      0.664975   |  -84.7 |  150.1 |    0.44 |                |
| 11 | B          |          111 | TRP            | nonpolar   | sheet     | E         |      0.555066   | -126.2 |  173.9 |    0.53 |                |
| 12 | B          |          112 | LYS            | positive   | sheet     | E         |      0.434146   | -139.2 |  122.6 |    0.46 |                |
| 13 | B          |          113 | THR            | polar      | sheet     | E         |      0.577465   |  -69.8 |  112.2 |    0.4  |                |
| 14 | B          |          114 | CYS            | polar      | loop      | -         |      0.0666667  |  -66.1 |  144.5 |    0.42 |                |
| 15 | B          |          115 | PRO            | nonpolar   | loop      | -         |      0.375      |  -76.9 |  176.1 |    0.52 |                |
| 16 | B          |          116 | LYS            | positive   | loop      | T         |      0.95122    |  -70.1 |  132.1 |    0.6  |                |
| 17 | B          |          117 | GLY            | nonpolar   | loop      | T         |      0.619048   |  106.9 |  -21.3 |    0.6  |                |
| 18 | B          |          118 | LYS            | positive   | loop      | -         |      0.17561    | -109.1 |   94.5 |    0.51 |                |
| 19 | B          |          119 | ASN            | polar      | loop      | -         |      0.738854   | -107.5 |   30   |    0.56 |                |
| 20 | B          |          120 | LEU            | nonpolar   | sheet     | E         |      0.262195   | -145.5 |  145.8 |    0.41 |                |
| 21 | B          |          121 | CYS            | polar      | sheet     | E         |      0          |  -98.4 |  150   |    0.34 |                |
| 22 | B          |          122 | TYR            | polar      | sheet     | E         |      0.0225225  | -145   |  150.2 |    0.38 |                |
| 23 | B          |          123 | LYS            | positive   | sheet     | E         |      0.229268   | -129   |  136.6 |    0.42 |                |
| 24 | B          |          124 | MET            | nonpolar   | sheet     | E         |      0.0106383  | -115.3 |  125   |    0.37 |                |
| 25 | B          |          125 | THR            | polar      | sheet     | E         |      0.084507   | -138   |  159.7 |    0.39 |                |
| 26 | B          |          126 | MET            | nonpolar   | sheet     | E         |      0.0797872  |  -68.4 |  151   |    0.45 |                |
| 27 | B          |          127 | ARG            | positive   | loop      | T         |      0.145161   |  -60.7 |  -25.4 |    0.47 |                |
| 28 | B          |          128 | ALA            | nonpolar   | loop      | T         |      0.650943   |  -97   |  -11.3 |    0.59 |                |
| 29 | B          |          129 | ALA            | nonpolar   | sheet     | E         |      0.292453   | -129.2 |   85.3 |    0.58 |                |
| 30 | B          |          130 | PRO            | nonpolar   | sheet     | E         |      0.558824   |  -72.4 |  -11.8 |    0.55 |                |
| 31 | B          |          131 | MET            | nonpolar   | sheet     | E         |      0.87234    | -102.7 |   11.5 |    0.65 |                |
| 32 | B          |          132 | VAL            | nonpolar   | sheet     | E         |      0.647887   | -123   |   90.9 |    0.53 |                |
| 33 | B          |          133 | PRO            | nonpolar   | sheet     | E         |      0.316176   |  -67   |  135.5 |    0.45 |                |
| 34 | B          |          134 | VAL            | nonpolar   | sheet     | E         |      0.204225   |  -97.4 |  -42.9 |    0.39 |                |
| 35 | B          |          135 | LYS            | positive   | sheet     | E         |      0.165854   | -135.1 |  150.8 |    0.41 |                |
| 36 | B          |          136 | ARG            | positive   | sheet     | E         |      0.233871   | -165.1 |  118.4 |    0.4  |                |
| 37 | B          |          137 | GLY            | nonpolar   | sheet     | E         |      0.0595238  |  167.3 | -156.3 |    0.36 |                |
| 38 | B          |          138 | CYS            | polar      | sheet     | E         |      0.118519   |  -77.9 |  155.3 |    0.38 |                |
| 39 | B          |          139 | ILE            | nonpolar   | sheet     | E         |      0.0295858  | -162.2 |  157.9 |    0.42 |                |
| 40 | B          |          140 | ASP            | negative   | loop      | S         |      0.595092   |  -88.3 |  -51.2 |    0.55 |                |
| 41 | B          |          141 | VAL            | nonpolar   | loop      | S         |      0.683099   | -121.8 |  140.2 |    0.49 |                |
| 42 | B          |          142 | CYS            | polar      | loop      | P         |      0.325926   |  -69.6 |  125.3 |    0.46 |                |
| 43 | B          |          143 | PRO            | nonpolar   | loop      | P         |      0.191176   |  -68.8 |  143.1 |    0.42 |                |
| 44 | B          |          144 | LYS            | positive   | loop      | P         |      0.346341   |  -72.8 |  135.3 |    0.48 |                |
| 45 | B          |          145 | SER            | polar      | loop      | P         |      0.0692308  |  -65.4 |  156.8 |    0.38 |                |
| 46 | B          |          146 | SER            | polar      | loop      | -         |      0.2        | -125.9 | -170.1 |    0.36 |                |
| 47 | B          |          147 | LEU            | nonpolar   | loop      | S         |      0.00609756 |  -66.9 |  -25.5 |    0.39 |                |
| 48 | B          |          148 | LEU            | nonpolar   | loop      | S         |      0          | -113   |  -18.6 |    0.36 |                |
| 49 | B          |          149 | ILE            | nonpolar   | sheet     | E         |      0.00591716 | -125.3 |  135.9 |    0.35 |                |
| 50 | B          |          150 | LYS            | positive   | sheet     | E         |      0.385366   | -113.9 |  132.1 |    0.46 |                |
| 51 | B          |          151 | TYR            | polar      | sheet     | E         |      0.0945946  | -113.1 |  132.1 |    0.4  |                |
| 52 | B          |          152 | MET            | nonpolar   | sheet     | E         |      0.430851   | -119.6 |  125.4 |    0.44 |                |
| 53 | B          |          153 | CYS            | polar      | sheet     | E         |      0.274074   | -123.2 |  151.6 |    0.41 |                |
| 54 | B          |          154 | CYS            | polar      | sheet     | E         |      0.162963   | -161.3 |  163.1 |    0.36 |                |
| 55 | B          |          155 | ASN            | polar      | loop      | -         |      0.732484   | -129.7 |   30   |    0.42 |                |
| 56 | B          |          156 | THR            | polar      | loop      | S         |      0.577465   | -120.9 |  167.6 |    0.46 |                |
| 57 | B          |          157 | ASP            | negative   | loop      | T         |      0.564417   |  -66.2 |  128.8 |    0.47 |                |
| 58 | B          |          158 | LYS            | positive   | loop      | T         |      0.468293   |   57.4 |   31.4 |    0.46 |                |
| 59 | B          |          159 | CYS            | polar      | loop      | -         |      0.266667   |  -80.6 |  -26.6 |    0.36 |                |
| 60 | B          |          160 | ASN            | polar      | loop      | -         |      0.0382166  |  -87.8 |  360   |    0.38 |                |

In [28]:
from Bio.PDB import PDBParser
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Data import IUPACData
from Bio.SeqUtils import ProtParamData

# 3-letter to 1-letter code map
three_to_one = IUPACData.protein_letters_3to1

# Load PDB
pdb_file = "./../target/9bk6.pdb"
parser = PDBParser()
structure = parser.get_structure("9bk6", pdb_file)

# Extract sequence and map residues
residue_list = []
sequence = ""

for chain in structure[0]:  # First model
    if chain.id == 'F':
        for residue in chain:
            resname = residue.get_resname().strip()
            try:
                one_letter = three_to_one[resname.capitalize()]
                sequence += one_letter
                residue_list.append(residue)
            except KeyError:
                continue  # Skip non-standard residues

print("Sequence:", sequence)
print("Length:", len(sequence))

# Hydropathy analysis with window
window = 9
pa = ProteinAnalysis(sequence)
hydropathy_values = pa.protein_scale(param_dict=ProtParamData.kd, window=window)

# Align hydropathy values to the residue list
offset = window // 2  # Values are centered in the window

for i, residue in enumerate(residue_list):
    resname = residue.get_resname()
    resnum = residue.get_id()[1]
    if offset <= i < len(residue_list) - offset:
        hydro = hydropathy_values[i - offset]
        print(f"Residue {resname} {resnum:>3}: Hydropathy = {hydro:.2f}")
    else:
        print(f"Residue {resname} {resnum:>3}: Hydropathy = NA")


Sequence: 
Length: 0


/opt/miniconda3/envs/alphafold/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1557.
  warnings.warn(
/opt/miniconda3/envs/alphafold/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 1590.
  warnings.warn(


In [30]:
import pandas as pd
from Bio.PDB import PDBParser
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Data import IUPACData
from Bio.SeqUtils import ProtParamData

def calculate_hydropathy_df(pdb_file, chain_id='F', window=9):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    
    three_to_one = IUPACData.protein_letters_3to1

    # Extract sequence and residues
    sequence = ""
    residue_info = []

    for chain in structure[0]:  # first model
        if chain.id == chain_id:
            for residue in chain:
                resname = residue.get_resname().strip()
                try:
                    one_letter = three_to_one[resname.capitalize()]
                    sequence += one_letter
                    residue_info.append({
                        'residue_name': resname,
                        'residue_id': residue.get_id()[1],
                        'chain_id': chain.id,
                        'one_letter': one_letter
                    })
                except KeyError:
                    continue  # skip non-standard residues

    # Calculate hydropathy
    analysed_seq = ProteinAnalysis(sequence)
    hydro_values = analysed_seq.protein_scale(param_dict=ProtParamData.kd, window=window)
    offset = window // 2

    # Add hydropathy to residue info
    for i in range(len(residue_info)):
        if offset <= i < len(residue_info) - offset:
            residue_info[i]['hydropathy'] = round(hydro_values[i - offset], 3)
        else:
            residue_info[i]['hydropathy'] = None

    # Add index
    for i, r in enumerate(residue_info):
        r['index'] = i + 1

    return pd.DataFrame(residue_info)

#  Run this
df_hydro = calculate_hydropathy_df("./../target/9bk6.pdb", chain_id='B', window=9)
print(df_hydro.head())

  residue_name  residue_id chain_id one_letter  hydropathy  index
0          LEU         101        B          L         NaN      1
1          LYS         102        B          K         NaN      2
2          CYS         103        B          C         NaN      3
3          ASN         104        B          N         NaN      4
4          GLN         105        B          Q       0.056      5


In [31]:
# Ensure consistent residue name format
df_hydro = df_hydro[['chain_id', 'residue_id', 'residue_name', 'hydropathy']]
df_hydro['residue_name'] = df_hydro['residue_name'].str.upper()

# Merge only hydropathy into df_target
df_merged = df_target.merge(
    df_hydro,
    on=['chain_id', 'residue_id', 'residue_name'],
    how='left'  # keep all rows from df_target
)

# Get list of columns
cols = df_merged.columns.tolist()
# Find the index of 'accessibility' column
access_idx = cols.index('accessibility')
# Remove 'hydropathy' from its current location
cols.remove('hydropathy')
# Insert 'hydropathy' right after 'accessibility'
cols.insert(access_idx + 1, 'hydropathy')
# Reorder dataframe
df_merged = df_merged[cols]

# df_merged = df_merged.sort_values(by=['accessibility', 'hydropathy'], ascending=[False, False]).reset_index(drop=True)

# Display result
from IPython.display import display, Markdown
display(Markdown(df_merged.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |    psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|-------:|--------:|:---------------|
| B          |          101 | LEU            | nonpolar   | loop      | -         |      0.371951   |      nan     |  360   |  128.4 |    0.41 |                |
| B          |          102 | LYS            | positive   | sheet     | E         |      0.37561    |      nan     | -112.8 |  127.3 |    0.47 |                |
| B          |          103 | CYS            | polar      | sheet     | E         |      0          |      nan     | -119.8 |  138.6 |    0.33 |                |
| B          |          104 | ASN            | polar      | sheet     | E         |      0.171975   |      nan     |  -63.4 |  141.3 |    0.36 |                |
| B          |          105 | GLN            | polar      | loop      | -         |      0.318182   |        0.056 |  -92.2 |  168.9 |    0.39 |                |
| B          |          106 | LEU            | nonpolar   | loop      | S         |      0.45122    |       -0.056 |  -62.8 |  -30.9 |    0.37 |                |
| B          |          107 | ILE            | nonpolar   | loop      | S         |      0.698225   |        0.278 |  -97.5 |  107.8 |    0.43 |                |
| B          |          108 | PRO            | nonpolar   | loop      | T         |      0.676471   |       -0.433 |  -70.4 |  155.3 |    0.51 |                |
| B          |          109 | PRO            | nonpolar   | loop      | T         |      0.772059   |       -0.122 |  -95.9 |   29.2 |    0.5  |                |
| B          |          110 | PHE            | nonpolar   | loop      | -         |      0.664975   |        0.544 |  -84.7 |  150.1 |    0.44 |                |
| B          |          111 | TRP            | nonpolar   | sheet     | E         |      0.555066   |       -0.056 | -126.2 |  173.9 |    0.53 |                |
| B          |          112 | LYS            | positive   | sheet     | E         |      0.434146   |       -0.989 | -139.2 |  122.6 |    0.46 |                |
| B          |          113 | THR            | polar      | sheet     | E         |      0.577465   |       -0.856 |  -69.8 |  112.2 |    0.4  |                |
| B          |          114 | CYS            | polar      | loop      | -         |      0.0666667  |       -1.111 |  -66.1 |  144.5 |    0.42 |                |
| B          |          115 | PRO            | nonpolar   | loop      | -         |      0.375      |       -1.811 |  -76.9 |  176.1 |    0.52 |                |
| B          |          116 | LYS            | positive   | loop      | T         |      0.95122    |       -1.289 |  -70.1 |  132.1 |    0.6  |                |
| B          |          117 | GLY            | nonpolar   | loop      | T         |      0.619048   |       -0.578 |  106.9 |  -21.3 |    0.6  |                |
| B          |          118 | LYS            | positive   | loop      | -         |      0.17561    |       -0.644 | -109.1 |   94.5 |    0.51 |                |
| B          |          119 | ASN            | polar      | loop      | -         |      0.738854   |       -1.356 | -107.5 |   30   |    0.56 |                |
| B          |          120 | LEU            | nonpolar   | sheet     | E         |      0.262195   |       -0.967 | -145.5 |  145.8 |    0.41 |                |
| B          |          121 | CYS            | polar      | sheet     | E         |      0          |       -0.611 |  -98.4 |  150   |    0.34 |                |
| B          |          122 | TYR            | polar      | sheet     | E         |      0.0225225  |       -0.356 | -145   |  150.2 |    0.38 |                |
| B          |          123 | LYS            | positive   | sheet     | E         |      0.229268   |       -0.422 | -129   |  136.6 |    0.42 |                |
| B          |          124 | MET            | nonpolar   | sheet     | E         |      0.0106383  |        0.167 | -115.3 |  125   |    0.37 |                |
| B          |          125 | THR            | polar      | sheet     | E         |      0.084507   |       -0.056 | -138   |  159.7 |    0.39 |                |
| B          |          126 | MET            | nonpolar   | sheet     | E         |      0.0797872  |       -0.511 |  -68.4 |  151   |    0.45 |                |
| B          |          127 | ARG            | positive   | loop      | T         |      0.145161   |       -0.156 |  -60.7 |  -25.4 |    0.47 |                |
| B          |          128 | ALA            | nonpolar   | loop      | T         |      0.650943   |        0.744 |  -97   |  -11.3 |    0.59 |                |
| B          |          129 | ALA            | nonpolar   | sheet     | E         |      0.292453   |        0.356 | -129.2 |   85.3 |    0.58 |                |
| B          |          130 | PRO            | nonpolar   | sheet     | E         |      0.558824   |        0.9   |  -72.4 |  -11.8 |    0.55 |                |
| B          |          131 | MET            | nonpolar   | sheet     | E         |      0.87234    |        0.256 | -102.7 |   11.5 |    0.65 |                |
| B          |          132 | VAL            | nonpolar   | sheet     | E         |      0.647887   |        0.256 | -123   |   90.9 |    0.53 |                |
| B          |          133 | PRO            | nonpolar   | sheet     | E         |      0.316176   |        0.011 |  -67   |  135.5 |    0.45 |                |
| B          |          134 | VAL            | nonpolar   | sheet     | E         |      0.204225   |        0.089 |  -97.4 |  -42.9 |    0.39 |                |
| B          |          135 | LYS            | positive   | sheet     | E         |      0.165854   |        0.767 | -135.1 |  150.8 |    0.41 |                |
| B          |          136 | ARG            | positive   | sheet     | E         |      0.233871   |        0.167 | -165.1 |  118.4 |    0.4  |                |
| B          |          137 | GLY            | nonpolar   | sheet     | E         |      0.0595238  |        0.167 |  167.3 | -156.3 |    0.36 |                |
| B          |          138 | CYS            | polar      | sheet     | E         |      0.118519   |        0.622 |  -77.9 |  155.3 |    0.38 |                |
| B          |          139 | ILE            | nonpolar   | sheet     | E         |      0.0295858  |       -0.022 | -162.2 |  157.9 |    0.42 |                |
| B          |          140 | ASP            | negative   | loop      | S         |      0.595092   |       -0.022 |  -88.3 |  -51.2 |    0.55 |                |
| B          |          141 | VAL            | nonpolar   | loop      | S         |      0.683099   |        0.389 | -121.8 |  140.2 |    0.49 |                |
| B          |          142 | CYS            | polar      | loop      | P         |      0.325926   |        0.344 |  -69.6 |  125.3 |    0.46 |                |
| B          |          143 | PRO            | nonpolar   | loop      | P         |      0.191176   |        0.489 |  -68.8 |  143.1 |    0.42 |                |
| B          |          144 | LYS            | positive   | loop      | P         |      0.346341   |        0.411 |  -72.8 |  135.3 |    0.48 |                |
| B          |          145 | SER            | polar      | loop      | P         |      0.0692308  |        1.3   |  -65.4 |  156.8 |    0.38 |                |
| B          |          146 | SER            | polar      | loop      | -         |      0.2        |        0.4   | -125.9 | -170.1 |    0.36 |                |
| B          |          147 | LEU            | nonpolar   | loop      | S         |      0.00609756 |       -0.022 |  -66.9 |  -25.5 |    0.39 |                |
| B          |          148 | LEU            | nonpolar   | loop      | S         |      0          |        0.367 | -113   |  -18.6 |    0.36 |                |
| B          |          149 | ILE            | nonpolar   | sheet     | E         |      0.00591716 |        1.078 | -125.3 |  135.9 |    0.35 |                |
| B          |          150 | LYS            | positive   | sheet     | E         |      0.385366   |        1.444 | -113.9 |  132.1 |    0.46 |                |
| B          |          151 | TYR            | polar      | sheet     | E         |      0.0945946  |        1.144 | -113.1 |  132.1 |    0.4  |                |
| B          |          152 | MET            | nonpolar   | sheet     | E         |      0.430851   |        0.644 | -119.6 |  125.4 |    0.44 |                |
| B          |          153 | CYS            | polar      | sheet     | E         |      0.274074   |       -0.167 | -123.2 |  151.6 |    0.41 |                |
| B          |          154 | CYS            | polar      | sheet     | E         |      0.162963   |       -1.1   | -161.3 |  163.1 |    0.36 |                |
| B          |          155 | ASN            | polar      | loop      | -         |      0.732484   |       -0.389 | -129.7 |   30   |    0.42 |                |
| B          |          156 | THR            | polar      | loop      | S         |      0.577465   |       -0.633 | -120.9 |  167.6 |    0.46 |                |
| B          |          157 | ASP            | negative   | loop      | T         |      0.564417   |      nan     |  -66.2 |  128.8 |    0.47 |                |
| B          |          158 | LYS            | positive   | loop      | T         |      0.468293   |      nan     |   57.4 |   31.4 |    0.46 |                |
| B          |          159 | CYS            | polar      | loop      | -         |      0.266667   |      nan     |  -80.6 |  -26.6 |    0.36 |                |
| B          |          160 | ASN            | polar      | loop      | -         |      0.0382166  |      nan     |  -87.8 |  360   |    0.38 |                |

In [32]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.7)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| B          |          109 | PRO            | nonpolar   | loop      | T         |        0.772059 |       -0.122 |  -95.9 |  29.2 |    0.5  |                |
| B          |          116 | LYS            | positive   | loop      | T         |        0.95122  |       -1.289 |  -70.1 | 132.1 |    0.6  |                |
| B          |          119 | ASN            | polar      | loop      | -         |        0.738854 |       -1.356 | -107.5 |  30   |    0.56 |                |
| B          |          131 | MET            | nonpolar   | sheet     | E         |        0.87234  |        0.256 | -102.7 |  11.5 |    0.65 |                |
| B          |          155 | ASN            | polar      | loop      | -         |        0.732484 |       -0.389 | -129.7 |  30   |    0.42 |                |

In [33]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.7) & (df_merged['hydropathy'] > 0.4)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   | residue_id   | residue_name   | property   | ss_type   | ss_code   | accessibility   | hydropathy   | phi   | psi   | plddt   | is_interface   |
|------------|--------------|----------------|------------|-----------|-----------|-----------------|--------------|-------|-------|---------|----------------|

In [34]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.6) & (df_merged['hydropathy'] > 0.3)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| B          |          110 | PHE            | nonpolar   | loop      | -         |        0.664975 |        0.544 |  -84.7 | 150.1 |    0.44 |                |
| B          |          128 | ALA            | nonpolar   | loop      | T         |        0.650943 |        0.744 |  -97   | -11.3 |    0.59 |                |
| B          |          141 | VAL            | nonpolar   | loop      | S         |        0.683099 |        0.389 | -121.8 | 140.2 |    0.49 |                |

In [35]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.5) & (df_merged['hydropathy'] > 0.3)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| B          |          110 | PHE            | nonpolar   | loop      | -         |        0.664975 |        0.544 |  -84.7 | 150.1 |    0.44 |                |
| B          |          128 | ALA            | nonpolar   | loop      | T         |        0.650943 |        0.744 |  -97   | -11.3 |    0.59 |                |
| B          |          130 | PRO            | nonpolar   | sheet     | E         |        0.558824 |        0.9   |  -72.4 | -11.8 |    0.55 |                |
| B          |          141 | VAL            | nonpolar   | loop      | S         |        0.683099 |        0.389 | -121.8 | 140.2 |    0.49 |                |

## Detecting Inter-Chain Contacts
To determine whether a particular chain (e.g., 'F') makes contact with other chains in a PDB structure, you can check for interatomic distances between residues in chain 'F' and residues in all other chains. If any atoms are closer than a distance threshold (commonly ~5 Å), it's considered a contact.

In [38]:
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

def get_chain_contacts(pdb_file, target_chain_id='F', distance_threshold=3.5):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    model = structure[0]  # Use the first model
    
    # Collect atoms from chain F and other chains
    target_atoms = []
    other_atoms = []
    
    for chain in model:
        for residue in chain:
            if not is_aa(residue):
                continue
            for atom in residue:
                if chain.id == target_chain_id:
                    target_atoms.append(atom)
                else:
                    other_atoms.append(atom)

    # Search for contacts using NeighborSearch
    ns = NeighborSearch(other_atoms)
    contacts = set()

    for atom in target_atoms:
        neighbors = ns.search(atom.coord, distance_threshold)
        for neighbor in neighbors:
            neighbor_chain = neighbor.get_parent().get_parent().id
            if neighbor_chain != target_chain_id:
                contacts.add(neighbor_chain)

    return list(contacts)

pdb_file = "./../target/9bk6.pdb"
contacting_chains = get_chain_contacts(pdb_file, target_chain_id='B')
print(f"Chain 'F' contacts: {contacting_chains}")

Chain 'F' contacts: ['A']


### Residue-Level Contact Map
identify residue-residue contacts between chain 'F' and other chains in a PDB file. This means listing which residue in chain 'F' is close to which residue in other chains, based on a distance cutoff (usually ~5.0 Å).

In [39]:
from Bio.PDB import PDBParser, NeighborSearch, is_aa

def get_chain_residue_contacts(pdb_file, target_chain_id='F', distance_threshold=5.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    model = structure[0]

    target_residues = [res for res in model[target_chain_id] if is_aa(res)]

    # Build list of all atoms (excluding the target chain) and map them to residues
    other_atoms = []
    atom_to_residue = {}
    for chain in model:
        if chain.id == target_chain_id:
            continue
        for residue in chain:
            if not is_aa(residue):
                continue
            for atom in residue:
                other_atoms.append(atom)
                atom_to_residue[atom] = residue

    # Build neighbor search index
    ns = NeighborSearch(other_atoms)

    # Store residue-residue contacts
    contacts = []

    for res_f in target_residues:
        for atom in res_f:
            neighbors = ns.search(atom.coord, distance_threshold)
            for neighbor in neighbors:
                res_other = atom_to_residue[neighbor]
                contact = {
                    'F_residue_name': res_f.get_resname(),
                    'F_residue_id': res_f.get_id()[1],
                    'contact_chain': res_other.get_parent().id,
                    'contact_residue_name': res_other.get_resname(),
                    'contact_residue_id': res_other.get_id()[1]
                }
                contacts.append(contact)

    # Remove duplicates
    contacts = [dict(t) for t in {tuple(d.items()) for d in contacts}]
    return contacts



contacts = get_chain_residue_contacts("./../target/9bk6.pdb", target_chain_id='B', distance_threshold=3.5)

import pandas as pd
df_contacts = pd.DataFrame(contacts)
df_contacts = df_contacts.sort_values(by="F_residue_id")
display(df_contacts)

,F_residue_name,F_residue_id,contact_chain,contact_residue_name,contact_residue_id
0,LYS,118,A,ASP,61
12,TYR,122,A,PRO,64
10,ARG,127,A,GLU,14
11,ARG,127,A,TYR,92
9,ALA,128,A,ARG,42
1,SER,145,A,TYR,92
7,SER,145,A,GLN,96
13,SER,146,A,GLU,55
2,LEU,147,A,GLU,55
4,LEU,147,A,TYR,21


In [42]:
f_residue_string = ",".join(df_contacts["F_residue_id"].apply(lambda x: f"B{x}").drop_duplicates())
print(f_residue_string)

B118,B122,B127,B128,B145,B146,B147,B148,B149,B151


In [78]:
contacts = get_chain_residue_contacts("./../out/bindcraft/snake-venom-binder/2506192105/Accepted/1yi5_l105_s971412_mpnn2_model2.pdb", target_chain_id='B', distance_threshold=3)
df_contacts_target = pd.DataFrame(contacts)
display(df_contacts_target)

,F_residue_name,F_residue_id,contact_chain,contact_residue_name,contact_residue_id
0,VAL,93,A,ARG,36
1,TRP,97,A,PRO,46
2,HIS,94,A,ASP,38
3,TYR,86,A,ARG,33
4,GLN,101,A,ALA,42
5,TYR,59,A,ILE,32
6,SER,52,A,ALA,28
7,TYR,86,A,PHE,29
8,SER,52,A,PHE,29
9,ARG,98,A,TYR,21


### Detect Disulfide Bonds in a Chain

In [41]:
from Bio.PDB import PDBParser, is_aa
from Bio.PDB.NeighborSearch import NeighborSearch

def find_disulfide_bonds(pdb_file, chain_id='F', distance_cutoff=2.2):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("struct", pdb_file)
    model = structure[0]
    chain = model[chain_id]
    
    # Get all CYS SG atoms
    sg_atoms = []
    residues = []
    
    for residue in chain:
        if is_aa(residue) and residue.get_resname() == "CYS":
            if "SG" in residue:
                sg_atoms.append(residue["SG"])
                residues.append(residue)

    # Search for SG-SG pairs within cutoff
    disulfides = []
    for i in range(len(sg_atoms)):
        for j in range(i + 1, len(sg_atoms)):
            dist = sg_atoms[i] - sg_atoms[j]
            if dist <= distance_cutoff:
                disulfides.append((residues[i].get_id()[1], residues[j].get_id()[1]))

    return disulfides

# pdb_file = "./../target/1yi5.pdb"
# disulfide_bonds = find_disulfide_bonds(pdb_file, chain_id='F')
# print("Disulfide bonds (residue ID pairs):", disulfide_bonds)

# pdb_file = "./../target/7z14.pdb"
# disulfide_bonds = find_disulfide_bonds(pdb_file, chain_id='F')
# print("Disulfide bonds (residue ID pairs):", disulfide_bonds)

pdb_file = "./../target/9bk6.pdb"
disulfide_bonds = find_disulfide_bonds(pdb_file, chain_id='B')
print("Disulfide bonds (residue ID pairs):", disulfide_bonds)


Disulfide bonds (residue ID pairs): [(103, 121), (114, 138), (142, 153), (154, 159)]
